# MPC IP Driver
MPC con ADMM. Inicialización con matrices de entrada, luego cada iteración se ingresa x0 y se obtiene u0.
Interfaz axilite, sin partition.

In [1]:
from pynq import Overlay

In [2]:
overlay = Overlay('/home/xilinx/pynq/overlays/mpc/mpc_baseline.bit')
overlay?

In [3]:
mpc_ip = overlay.mpc_sparse_admm_iter_0
mpc_ip?

In [4]:
mpc_ip.register_map

/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/registers.py:398: UserWarning: Unsupported register size 256 for register Memory_M_hat_0
  warnings.warn(
/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/registers.py:398: UserWarning: Unsupported register size 256 for register Memory_M_hat_1
  warnings.warn(
/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/registers.py:398: UserWarning: Unsupported register size 256 for register Memory_M_hat_2
  warnings.warn(
/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/registers.py:398: UserWarning: Unsupported register size 256 for register Memory_M_hat_3
  warnings.warn(
/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/registers.py:398: UserWarning: Unsupported register size 256 for register Memory_M_hat_4
  warnings.warn(
/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/registers.py:398: UserWarning: Unsupported register size 256 for register Memory_M_hat_5
  warnings.warn(
/usr

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  r0 = Register(r0=write-only),
  Memory_x0 = Register(value=0),
  Memory_x1 = Register(value=0),
  u0 = Register(u0=0),
  u0_ctrl = Register(u0_ap_vld=0, RESERVED=0),
  Memory_RhoMt_neg_0 = Register(value=0),
  Memory_RhoMt_neg_1 = Register(value=0),
  Memory_RhoMt_neg_2 = Register(value=0),
  Memory_RhoMt_neg_3 = Register(value=0),
  Memory_RhoMt_neg_4 = Register(value=0),
  Memory_RhoMt_neg_5 = Register(value=0),
  Memory_RhoMt_neg_6 = Register(value=0),
  Memory_RhoMt_neg_7 = Register(value=0),
  Memory_RhoMt_neg_8 = Register(value=0),
  Memory_RhoMt_neg_9 = Register(value=0),
  Memory_RhoMt_neg_10 = Register(value=0),
  Memory_RhoMt_neg_11 = Register(value=0),
 


### Driver especializado

In [10]:
from pynq import DefaultIP
import struct

class mpcDriver(DefaultIP):
    
    # IP Register Map
    CTRL_REG = 0x0000
    GIE_REG = 0x0004
    INTE_REG = 0x0008
    INTS_REG = 0x000C
    r0_REG = 0x0010
    x0_REG = 0x0018
    x1_REG = 0x0020
    u0_REG = 0x0028
    u0_CTRL_REG = 0x002c
    hqp_REG = 0x1a40
    Rinv_REG = [0x1a80, 0x1ac0, 0x1b00, 0x1b40, 0x1b80, 0x1bc0, 0x1c00, 0x1c40, 0x1c80, 0x1cc0, 0x1d00, 0x1d40, 0x1d80, 0x1dc0]
    Mhat_REG = [0x0100, 0x0200, 0x0300, 0x0400, 0x0500, 0x0600, 0x0700, 0x0800, 0x0900, 0x0a00, 0x0b00, 0x0c00, 0x0d00, 0x0e00]
    RhoMtneg_REG = [0x0040, 0x0080, 0x00c0, 0x0f00, 0x0f40, 0x0f80, 0x0fc0, 0x1000, 0x1040, 0x1080, 0x10c0, 0x1100, 
                    0x1140, 0x1180, 0x11c0, 0x1200, 0x1240, 0x1280, 0x12c0, 0x1300, 0x1340, 0x1380, 0x13c0, 0x1400, 
                    0x1440, 0x1480, 0x14c0, 0x1500, 0x1540, 0x1580, 0x15c0, 0x1600, 0x1640, 0x1680, 0x16c0, 0x1700, 
                    0x1740, 0x1780, 0x17c0, 0x1800, 0x1840, 0x1880, 0x18c0, 0x1900, 0x1940, 0x1980, 0x19c0, 0x1a00]
    
    N_SYS = 2;
    N_QP = 14;
    M_QP = 48;
    
    def __init__(self, description):
        super().__init__(description=description)
        self.time_start = 0
        self.time_stop = 0

    bindto = ['xilinx.com:hls:mpc_sparse_admm_iteration:1.0']

    def is_ready(self):
        state = self.read(mpcDriver.CTRL_REG)
        return (state >> 3) & 0x01

    def is_done(self):
        state = self.read(mpcDriver.CTRL_REG)
        return (state >> 1) & 0x01

    def is_idle(self):
        state = self.read(mpcDriver.CTRL_REG)
        return (state >> 2) & 0x01

    def start(self):
        self.write(mpcDriver.CTRL_REG, 0x00000001)

    def write_Mhat(self, data):
        if (len(data)!=self.M_QP) or (len(data[0])!=self.N_QP) :
            printf("Size error")
        else:
            for col in range(0, self.N_QP):
                offset = mpcDriver.Mhat_REG[col]
                for row in range (0, self.M_QP):
                    word_item = struct.pack("f", data[row][col])
                    #print(f"ADDRESS     : {hex(offset)}")
                    #print(f"DATA (float): {data[row][col]}")
                    #print(f"DATA (hex)  : 0x{word_item.hex()}")
                    #print(f"DATA (uint) : {int.from_bytes(word_item, 'little')}")
                    self.write(offset, int.from_bytes(word_item,'little'))
                    offset = offset + 4
    
    def write_RhoMtneg(self, data):
        if (len(data)!=self.N_QP) or (len(data[0])!=self.M_QP) :
            printf("Size error")
        else:
            for col in range(0, self.M_QP):
                offset = mpcDriver.RhoMtneg_REG[col]
                for row in range(0, self.N_QP):
                    word_item = struct.pack("f", data[row][col])
                    self.write(offset, int.from_bytes(word_item,'little'))
                    offset = offset + 4
    
    def write_hqp(self, data):
        if (len(data)!=self.N_QP):
            printf("Size error")
        else:
            offset = mpcDriver.hqp_REG
            data_bytes = map(lambda x : struct.pack("f", x), data)
            for index, word_item in enumerate(data_bytes):
                self.write(offset, int.from_bytes(word_item,'little'))
                offset = offset + 4
    
    def write_Rinv(self, data):
        if (len(data)!=self.N_QP) or (len(data[0])!=self.N_QP) :
            printf("Size error")
        else:
            for col in range(0, self.N_QP):
                offset = mpcDriver.Rinv_REG[col]
                for row in range(0, self.N_QP):
                    word_item = struct.pack("f", data[row][col])
                    self.write(offset, int.from_bytes(word_item,'little'))
                    offset = offset + 4 

    def write_x0(self, data):
        if (len(data)!=self.N_SYS):
            printf("Size error")
        else:
            # START TIMING
            #self.time_start = time.time_ns()
            #self.time_start = time.perf_counter_ns()
            offset = mpcDriver.x0_REG
            data_bytes = map(lambda x : struct.pack("f", x), data)
            for index, word_item in enumerate(data_bytes):
                self.write(offset, int.from_bytes(word_item,'little'))
                offset = offset + 4
    
    def read_u0(self):
        #STOP TIMING
        #self.time_stop = time.time_ns()
        #self.time_stop = time.perf_counter_ns()
        if (self.read(mpcDriver.u0_CTRL_REG)):
            u0 = self.read(mpcDriver.u0_REG)
            print(u0)
            return struct.unpack("f", u0.to_bytes(4,'little'))[0]
        return 1
    
#    def write_brams(self, data):
#        # START TIMING
#        #self.time_start = time.time_ns()
#        #self.time_start = time.perf_counter_ns()
#        for i in range(0,3):
#            data_bytes = map(lambda x : struct.pack("f", x), data[i])
#            for idx, word_item in enumerate(data_bytes):
#                print(f"ADDRESS     : {hex(MyDriver.base_offset[i] + (idx * 4))}")
#                print(f"DATA (float): {data[i][idx]}")
#                print(f"DATA (hex)  : 0x{word_item.hex()}")
#                print(f"DATA (uint) : {int.from_bytes(word_item, 'little')}")
#                self.write(MyDriver.base_offset[i] + (idx * 4), int.from_bytes(word_item,'little'))


#    def read_result(self):
#        #STOP TIMING
#        #self.time_stop = time.time_ns()
#        #self.time_stop = time.perf_counter_ns()
#        if (self.read(MyDriver.R_CTRL_REG)):
#            R_out = self.read(MyDriver.R_REG)
#            print(R_out)
#            for idx in range(0,4):
#                data_out = self.read(MyDriver.D_REG + (idx * 4))
#                print(data_out)
#            #return tuple(map(lambda x : struct.unpack("f", x.to_bytes(4,'little'))[0], data_out))
#        #return (0, 1)

    def get_extime(self):
        return self.time_stop-self.time_start

### Test

In [27]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pynq/overlays/mpc/mpc_baseline.bit')
mpc_ip = overlay.mpc_sparse_admm_iter_0

In [5]:
M_hat =  [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-1.0],        
          [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.9725, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0385, 0.0, 0.0, 0.0],
          [0.001, 1.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.9725, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0385, 0.0, 0.0],
          [0.0, 0.0, 0.001, 1.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.9725, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0385, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.001, 1.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9725, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0385],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
          [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [-0.9725, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.0385, 0.0, 0.0, 0.0],
          [-0.001,-1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0,-0.9725, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.0385, 0.0, 0.0],
          [0.0, 0.0,-0.001,-1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0,-0.9725, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.0385, 0.0],
          [0.0, 0.0, 0.0, 0.0,-0.001,-1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.9725, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0,-0.0385],
          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-0.001,-1.0, 0.0,-1.0, 0.0, 0.0, 0.0, 0.0]]

h_qp = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

R_inv = [[0.0031, 0.0000, 0.0012, 0.0000, 0.0004, 0.0000, 0.0002, 0.0000, 0.0001, 0.0000,-0.0001, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0030, 0.0000, 0.0011, 0.0000, 0.0004, 0.0000, 0.0002, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0012, 0.0000, 0.0035, 0.0000, 0.0013, 0.0000, 0.0005, 0.0000,-0.0003, 0.0000, 0.0001,-0.0001, 0.0000, 0.0000],
         [0.0000, 0.0011, 0.0000, 0.0035, 0.0000, 0.0013, 0.0000, 0.0005, 0.0000,-0.0001, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0004, 0.0000, 0.0013, 0.0000, 0.0036, 0.0000, 0.0014, 0.0000,-0.0007, 0.0000, 0.0000, 0.0001,-0.0001, 0.0000],
         [0.0000, 0.0004, 0.0000, 0.0013, 0.0000, 0.0035, 0.0000, 0.0013, 0.0000,-0.0003, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0002, 0.0000, 0.0005, 0.0000, 0.0014, 0.0000, 0.0038, 0.0000, 0.0018, 0.0000, 0.0000, 0.0000, 0.0001,-0.0001],
         [0.0000, 0.0002, 0.0000, 0.0005, 0.0000, 0.0013, 0.0000, 0.0033, 0.0000,-0.0008, 0.0000, 0.0000, 0.0000, 0.0000],
         [-0.0001,0.0000,-0.0003, 0.0000,-0.0007, 0.0000,-0.0018, 0.0000, 0.0049,-0.0001, 0.0000, 0.0000, 0.0000,-0.0001],
         [0.0000, 0.0000, 0.0000, 0.0001, 0.0000,-0.0003, 0.0000,-0.0008,-0.0001, 0.0022, 0.0000, 0.0000, 0.0000, 0.0000],
         [-0.0001,0.0000, 0.0001, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0079, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000,-0.0001, 0.0000, 0.0001, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0079, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000,-0.0001, 0.0000, 0.0001, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0079, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,-0.0001, 0.0000,-0.0001, 0.0000, 0.0000, 0.0000, 0.0000, 0.0079]]
    
RhoMt_neg = [[-62.9634, 0.0,62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634, 0.0,-61.2298,-0.0621, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 61.2298, 0.0621, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
             [0.0,-62.9634, 0.0, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634, 0.0,-62.9634,    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634,    0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
             [0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634, 0.0,-61.2298,-0.0621, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 61.2298, 0.0621, 0.0, 0.0, 0.0, 0.0],
             [0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634, 0.0,-62.9634,    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634,    0.0, 0.0, 0.0, 0.0],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634, 0.0,-61.2298,-0.0621, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 61.2298, 0.0621, 0.0, 0.0],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634, 0.0,-62.9634,    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634,    0.0, 0.0],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634, 0.0,-61.2298,-0.0621, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 61.2298, 0.0621],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634, 0.0,-62.9634,    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634, 0.0],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.9634],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-2.4226,-0.0012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.4266, 0.0012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-2.4226,-0.0012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.4266, 0.0012, 0.0, 0.0, 0.0, 0.0],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-2.4226,-0.0012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.4266, 0.0012, 0.0, 0.0],
             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-62.9634, 62.9634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-2.4226,-0.0012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.4266, 0.0012]]


In [14]:
R_inv = [[ 3.0803e-03,-8.6322e-07, 1.1617e-03, 4.3977e-07, 4.4042e-04, 2.9486e-07, 1.7305e-04,-2.6384e-07,-8.4035e-05, 1.3520e-06,-7.0511e-05,-2.6505e-05,-9.8146e-06,-3.2397e-06],
         [-8.6322e-07, 3.0221e-03,-7.1661e-07, 1.1491e-03,-5.8172e-07, 4.3430e-04,-7.6465e-07, 1.5726e-04, 1.6151e-06,-3.8731e-05,-3.1454e-08,-9.3781e-09,-1.3001e-08,-3.5802e-08],
         [ 1.1617e-03,-7.1661e-07, 3.5225e-03,-1.0081e-06, 1.3354e-03, 9.0587e-09, 5.2472e-04,-1.1204e-06,-2.5481e-04, 4.1784e-06, 9.2005e-05,-8.0366e-05,-2.9759e-05,-9.8232e-06],
         [ 4.3977e-07, 1.1491e-03,-1.0081e-06, 3.4564e-03,-1.3148e-06, 1.3063e-03,-2.1291e-06, 4.7303e-04, 4.7753e-06,-1.1650e-04,-1.0642e-08,-5.4387e-08,-4.8800e-08,-1.1089e-07],
         [ 4.4042e-04,-5.8172e-07, 1.3354e-03,-1.3148e-06, 3.6067e-03,-2.4893e-06, 1.4172e-03,-3.9364e-06,-6.8820e-04, 1.1509e-05, 3.4881e-05, 8.8750e-05,-8.0374e-05,-2.6530e-05],
         [ 2.9486e-07, 4.3430e-04, 9.0587e-09, 1.3063e-03,-2.4893e-06, 3.4951e-03,-5.2923e-06, 1.2656e-03, 1.2580e-05,-3.1169e-04, 6.1665e-09,-5.3781e-08,-1.5348e-07,-3.0425e-07],
         [ 1.7305e-04,-7.6465e-07, 5.2472e-04,-2.1291e-06, 1.4172e-03,-5.2923e-06, 3.7702e-03,-1.2909e-05,-1.8309e-03, 3.1219e-05, 1.3706e-05, 3.4872e-05, 9.1979e-05,-7.0580e-05],
         [-2.6384e-07, 1.5726e-04,-1.1204e-06, 4.7303e-04,-3.9364e-06, 1.2656e-03,-1.2909e-05, 3.3338e-03, 3.2637e-05,-8.2104e-04,-2.7118e-08,-9.4155e-08,-3.0922e-07,-8.2077e-07],
         [-8.4035e-05, 1.6151e-06,-2.5481e-04, 4.7753e-06,-6.8820e-04, 1.2580e-05,-1.8309e-03, 3.2637e-05, 4.8606e-03,-8.4461e-05,-6.6555e-06,-1.6934e-05,-4.4667e-05,-1.1843e-04],
         [ 1.3520e-06,-3.8731e-05, 4.1784e-06,-1.1650e-04, 1.1509e-05,-3.1169e-04, 3.1219e-05,-8.2104e-04,-8.4461e-05, 2.1579e-03, 1.0861e-07, 2.8254e-07, 7.6022e-07, 2.0545e-06],
         [-7.0511e-05,-3.1454e-08, 9.2005e-05,-1.0642e-08, 3.4881e-05, 6.1665e-09, 1.3706e-05,-2.7118e-08,-6.6555e-06, 1.0861e-07, 7.9292e-03,-2.0991e-06,-7.7730e-07,-2.5658e-07],
         [-2.6505e-05,-9.3781e-09,-8.0366e-05,-5.4387e-08, 8.8750e-05,-5.3781e-08, 3.4872e-05,-9.4155e-08,-1.6934e-05, 2.8254e-07,-2.0991e-06, 7.9295e-03,-1.9778e-06,-6.5283e-07],
         [-9.8146e-06,-1.3001e-08,-2.9759e-05,-4.8800e-08,-8.0374e-05,-1.5348e-07, 9.1979e-05,-3.0922e-07,-4.4667e-05, 7.6022e-07,-7.7730e-07,-1.9778e-06, 7.9296e-03,-1.7219e-06],
         [-3.2397e-06,-3.5802e-08,-9.8232e-06,-1.1089e-07,-2.6530e-05,-3.0425e-07,-7.0580e-05,-8.2077e-07,-1.1843e-04, 2.0545e-06,-2.5658e-07,-6.5283e-07,-1.7219e-06, 7.9303e-03]]

In [30]:
mpc_ip.write_Mhat(M_hat)
mpc_ip.write_hqp(h_qp)
mpc_ip.write_Rinv(R_inv)
mpc_ip.write_RhoMtneg(RhoMt_neg)

In [31]:
x0 = [3.0, -1.0]
mpc_ip.write_x0(x0)

In [32]:
mpc_ip.start()
mpc_ip.read(0x0000)

14

In [33]:
mpc_ip.read(0x0000)

4

In [34]:
mpc_ip.read(0x0000)

4

In [35]:
mpc_ip.read_u0()

3204985800


-0.5320401191711426

In [21]:
x0 = [2.89686, -0.997052]
mpc_ip.write_x0(x0)
mpc_ip.start()
mpc_ip.read(0x0000)

14

In [22]:
mpc_ip.read_u0()

3198000333


-0.3078369200229645

In [23]:
x0 = [2.80519, -0.994201]
mpc_ip.write_x0(x0)
mpc_ip.start()
mpc_ip.read(0x0000)

14

In [24]:
mpc_ip.read_u0()

3199650623


-0.3570193946361542

In [25]:
mpc_ip.start()
mpc_ip.read(0x0000)

14

In [26]:
mpc_ip.read_u0()

3198529202


-0.3235984444618225

In [29]:
mpc_ip.read(0x0028)

0